## Installing Necessary Libraries & Import Them

In [1]:
!pip install langchain langchain_community unstructured sentence_transformers tiktoken chromadb langchain_chroma langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install geopandas

In [3]:
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

## Load the Dataset

In [5]:
import geopandas as gpd
# Load the data
df = gpd.read_file('/content/merged_6month.gpkg')
df

,accident_id,street_name_left,number_of_cars_involved,time_series_left,accident_reason,number_of_injured,traffic_id,time_series_right,street_name_right,weather,number_of_cars,average_car_speed,traffic_status,day_name,date,point_geometry_wkt,geometry
0,0,طريق خريص,3,2023-01-01 06:00:00,Road Conditions,0,177,2023-01-01 06:00:00,طريق خريص,Clear,1464,101.28,Light,Sunday,None,POINT (46.8123571 24.7456475),"LINESTRING (46.81213 24.7455, 46.81236 24.74565)"
1,1,خريص,3,2023-01-01 11:00:00,Speeding,0,2329,2023-01-01 11:00:00,خريص,Cloudy,771,37.18,Heavy,Sunday,None,POINT (46.794121 24.7343353),"LINESTRING (46.79363 24.73404, 46.79412 24.73434)"
2,2,خريص,2,2023-01-01 14:00:00,Road Conditions,0,3732,2023-01-01 14:00:00,خريص,Rainy,360,31.73,Heavy,Sunday,None,POINT (46.7826574 24.7277724),"LINESTRING (46.78266 24.72777, 46.7817 24.72718)"
3,3,خريص,2,2023-01-01 16:00:00,Road Conditions,0,4768,2023-01-01 16:00:00,خريص,Clear,955,55.54,Moderate,Sunday,None,POINT (46.8631253 24.7938616),"LINESTRING (46.86313 24.79386, 46.86064 24.79053)"
4,4,طريق خريص,2,2023-01-02 18:00:00,Road Conditions,0,11458,2023-01-02 18:00:00,طريق خريص,Clear,731,32.44,Heavy,Monday,None,POINT (46.7998682 24.7378718),"LINESTRING (46.79987 24.73787, 46.80016 24.73804)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,1033,طريق مكة المكرمة,2,2023-06-29 18:00:00,Using Phone,5,941171,2023-06-29 18:00:00,طريق مكة المكرمة,Foggy,947,81.08,Moderate,Thursday,None,POINT (46.7569355 24.7114081),"LINESTRING (46.74937 24.70669, 46.75066 24.707..."
1034,1034,خريص,1,2023-06-30 23:00:00,Road Conditions,0,946019,2023-06-30 23:00:00,خريص,Clear,682,30.71,Heavy,Friday,None,POINT (46.7997335 24.7384061),"LINESTRING (46.79995 24.73854, 46.79973 24.738..."
1035,1035,مكة المكرمة,2,2023-06-30 18:00:00,Using Phone,0,943683,2023-06-30 18:00:00,مكة المكرمة,Clear,852,45.73,Moderate,Friday,None,POINT (46.6839201 24.6765555),"LINESTRING (46.6835 24.6764, 46.68392 24.67656)"
1036,1036,خريص,1,2023-06-30 20:00:00,Road Conditions,0,944434,2023-06-30 20:00:00,خريص,Clear,332,39.67,Heavy,Friday,None,POINT (46.7962479 24.7362403),"LINESTRING (46.79625 24.73624, 46.79449 24.73514)"


##translations


In [6]:

translations = {
    "day_name": {
        "Monday": "الإثنين",
        "Tuesday": "الثلاثاء",
        "Wednesday": "الأربعاء",
        "Thursday": "الخميس",
        "Friday": "الجمعة",
        "Saturday": "السبت",
        "Sunday": "الأحد"
    },
    "weather": {
        "Clear": "الطقس صافي",
        "Cloudy": "غائم",
        "Rainy": "ممطر",
        "Foggy": "ضبابي",
        "Stormy": "عاصف"
    },
    "traffic_status": {
        "Heavy": " مزدحم",
        "Moderate": " ازدحام معتدل",
        "Light": "لايوجد ازدحام"
    },
        "accident_reason": {
         "Road Conditions": "احوال الطريق",
         "Using Phone": "استخدام الهاتف",
         "Reckless Driving": "القيادة المتهورة",
         "Speeding": "السرعة",
         "Weather": "الطقس"
    }
}



def translate_to_arabic(category, term):
    if category in translations and term in translations[category]:
        return translations[category][term]
    else:
        return "ترجمة غير متوفرة"



In [7]:
df['day_name']= df['day_name'].apply(lambda x: translations['day_name'][x])
df['weather']= df['weather'].apply(lambda x: translations['weather'][x])
df['traffic_status']= df['traffic_status'].apply(lambda x: translations['traffic_status'][x])
df['accident_reason']= df['accident_reason'].apply(lambda x: translations['accident_reason'][x])

In [8]:
df

,accident_id,street_name_left,number_of_cars_involved,time_series_left,accident_reason,number_of_injured,traffic_id,time_series_right,street_name_right,weather,number_of_cars,average_car_speed,traffic_status,day_name,date,point_geometry_wkt,geometry
0,0,طريق خريص,3,2023-01-01 06:00:00,احوال الطريق,0,177,2023-01-01 06:00:00,طريق خريص,الطقس صافي,1464,101.28,لايوجد ازدحام,الأحد,None,POINT (46.8123571 24.7456475),"LINESTRING (46.81213 24.7455, 46.81236 24.74565)"
1,1,خريص,3,2023-01-01 11:00:00,السرعة,0,2329,2023-01-01 11:00:00,خريص,غائم,771,37.18,مزدحم,الأحد,None,POINT (46.794121 24.7343353),"LINESTRING (46.79363 24.73404, 46.79412 24.73434)"
2,2,خريص,2,2023-01-01 14:00:00,احوال الطريق,0,3732,2023-01-01 14:00:00,خريص,ممطر,360,31.73,مزدحم,الأحد,None,POINT (46.7826574 24.7277724),"LINESTRING (46.78266 24.72777, 46.7817 24.72718)"
3,3,خريص,2,2023-01-01 16:00:00,احوال الطريق,0,4768,2023-01-01 16:00:00,خريص,الطقس صافي,955,55.54,ازدحام معتدل,الأحد,None,POINT (46.8631253 24.7938616),"LINESTRING (46.86313 24.79386, 46.86064 24.79053)"
4,4,طريق خريص,2,2023-01-02 18:00:00,احوال الطريق,0,11458,2023-01-02 18:00:00,طريق خريص,الطقس صافي,731,32.44,مزدحم,الإثنين,None,POINT (46.7998682 24.7378718),"LINESTRING (46.79987 24.73787, 46.80016 24.73804)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,1033,طريق مكة المكرمة,2,2023-06-29 18:00:00,استخدام الهاتف,5,941171,2023-06-29 18:00:00,طريق مكة المكرمة,ضبابي,947,81.08,ازدحام معتدل,الخميس,None,POINT (46.7569355 24.7114081),"LINESTRING (46.74937 24.70669, 46.75066 24.707..."
1034,1034,خريص,1,2023-06-30 23:00:00,احوال الطريق,0,946019,2023-06-30 23:00:00,خريص,الطقس صافي,682,30.71,مزدحم,الجمعة,None,POINT (46.7997335 24.7384061),"LINESTRING (46.79995 24.73854, 46.79973 24.738..."
1035,1035,مكة المكرمة,2,2023-06-30 18:00:00,استخدام الهاتف,0,943683,2023-06-30 18:00:00,مكة المكرمة,الطقس صافي,852,45.73,ازدحام معتدل,الجمعة,None,POINT (46.6839201 24.6765555),"LINESTRING (46.6835 24.6764, 46.68392 24.67656)"
1036,1036,خريص,1,2023-06-30 20:00:00,احوال الطريق,0,944434,2023-06-30 20:00:00,خريص,الطقس صافي,332,39.67,مزدحم,الجمعة,None,POINT (46.7962479 24.7362403),"LINESTRING (46.79625 24.73624, 46.79449 24.73514)"


## Set Up a Directory for Markdown Files

In [9]:
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [10]:

for index, row in df.iterrows():
    traffic = row['traffic_status']

    # Convert all columns to string and concatenate them
    content = " ".join(f"**{col}:** {row[col]}" for col in df.columns if col != 'traffic')

    # Create markdown content
    markdown_content = f"# {traffic}\n\n{content}\n\n"

    with open(f'{directory}/{index}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

## Read Markdown Files from the Directory

In [11]:
markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)

## Split the Text into Chunks

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
documents = text_splitter.create_documents(markdown_texts)

## Initialize the Embedding Model & Create a Vector Store Using Chroma

Next, we initialize the embedding model using `SentenceTransformerEmbeddings`. We'll use the pre-trained model "all-MiniLM-L6-v2" for generating embeddings from the text chunks.

In [13]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(documents, embedding_function, persist_directory="./chroma_db")

<ipython-input-13-54f91d5a9530>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Define Persistent Directory for Chroma DB

In [14]:
PRESIST_DIRECTORY = '/content/chroma_db'
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
persist_directory = "./chroma_db"
db = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)

In [15]:
def query_chroma_db(query, db, top_k=10):
    docs = db.similarity_search(query)
    results = [doc.page_content for doc in docs]
    return results

##Create & Test the Retrieval with a Sample Query

In [16]:

PROMPT_TEMPLATE="""
السياق: {context}
السؤال: {question}
الإجابة:
 يرجى تقديم اجابة باللغة العربيه فقط و تتضمن ايام الاسبوع السبعه كلها
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

In [25]:
groq_api_key = "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"
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

In [26]:
MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

In [27]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=10)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = MODEL.invoke({"context": context_text, "question": query})

    return rag_response

In [36]:
response4 = query_rag("في يومم الخميس ماهي حاله طريق خريص")
response4



> Entering new LLMChain chain...
Prompt after formatting:

السياق: <h1>مزدحم</h1>
<p><strong>accident_id:</strong> 538 <strong>street_name_left:</strong> خريص <strong>number_of_cars_involved:</strong> 1 <strong>time_series_left:</strong> 2023-04-05 07:00:00 <strong>accident_reason:</strong> احوال الطريق <strong>number_of_injured:</strong> 1 <strong>traffic_id:</strong> 492488 <strong>time_series_right:</strong> 2023-04-05 07:00:00 <strong>street_name_right:</strong> خريص <strong>weather:</strong> ممطر <strong>number_of_cars:</strong> 770 <strong>average_car_speed:</strong> 25.34 <strong>traffic_status:</strong>  مزدحم <strong>day_name:</strong> الأربعاء <strong>date:</strong> None <strong>point_geometry_wkt:</strong> POINT (46.8631253 24.7938616) <strong>geometry:</strong> LINESTRING (46.8631253 24.7938616, 46.860642 24.7905276)</p>

<h1>مزدحم</h1>
<p><strong>accident_id:</strong> 778 <strong>street_name_left:</strong> خريص <strong>number_of_cars_involved:</strong> 1 <strong>time_ser

{'context': '<h1>مزدحم</h1>\n<p><strong>accident_id:</strong> 538 <strong>street_name_left:</strong> خريص <strong>number_of_cars_involved:</strong> 1 <strong>time_series_left:</strong> 2023-04-05 07:00:00 <strong>accident_reason:</strong> احوال الطريق <strong>number_of_injured:</strong> 1 <strong>traffic_id:</strong> 492488 <strong>time_series_right:</strong> 2023-04-05 07:00:00 <strong>street_name_right:</strong> خريص <strong>weather:</strong> ممطر <strong>number_of_cars:</strong> 770 <strong>average_car_speed:</strong> 25.34 <strong>traffic_status:</strong>  مزدحم <strong>day_name:</strong> الأربعاء <strong>date:</strong> None <strong>point_geometry_wkt:</strong> POINT (46.8631253 24.7938616) <strong>geometry:</strong> LINESTRING (46.8631253 24.7938616, 46.860642 24.7905276)</p>\n\n<h1>مزدحم</h1>\n<p><strong>accident_id:</strong> 778 <strong>street_name_left:</strong> خريص <strong>number_of_cars_involved:</strong> 1 <strong>time_series_left:</strong> 2023-05-15 15:00:00 <strong>accid

In [37]:
print(f'\n\nQuestion:\n{response4["question"]}\n\nAnswer: \n{response4["text"]}')



Question:
في يومم الخميس ماهي حاله طريق خريص

Answer: 
في يوم الخميس، حال الطريق خريص مزدحم، حسب المعلومات الموجودة في النص.


In [32]:
response2 = query_rag("what is most accident reason ?")
response2



> Entering new LLMChain chain...
Prompt after formatting:

السياق: <h1>مزدحم</h1>
<p><strong>accident_id:</strong> 463 <strong>street_name_left:</strong> مكة المكرمة <strong>number_of_cars_involved:</strong> 1 <strong>time_series_left:</strong> 2023-03-23 17:00:00 <strong>accident_reason:</strong> استخدام الهاتف <strong>number_of_injured:</strong> 0 <strong>traffic_id:</strong> 429679 <strong>time_series_right:</strong> 2023-03-23 17:00:00 <strong>street_name_right:</strong> مكة المكرمة <strong>weather:</strong> عاصف <strong>number_of_cars:</strong> 355 <strong>average_car_speed:</strong> 23.02 <strong>traffic_status:</strong>  مزدحم <strong>day_name:</strong> الخميس <strong>date:</strong> None <strong>point_geometry_wkt:</strong> POINT (46.7578008 24.7122812) <strong>geometry:</strong> LINESTRING (46.7583241 24.7126064, 46.7578008 24.7122812, 46.7567208 24.7116006)</p>

<h1>لايوجد ازدحام</h1>
<p><strong>accident_id:</strong> 133 <strong>street_name_left:</strong> مكة المكرمة <strong

{'context': '<h1>مزدحم</h1>\n<p><strong>accident_id:</strong> 463 <strong>street_name_left:</strong> مكة المكرمة <strong>number_of_cars_involved:</strong> 1 <strong>time_series_left:</strong> 2023-03-23 17:00:00 <strong>accident_reason:</strong> استخدام الهاتف <strong>number_of_injured:</strong> 0 <strong>traffic_id:</strong> 429679 <strong>time_series_right:</strong> 2023-03-23 17:00:00 <strong>street_name_right:</strong> مكة المكرمة <strong>weather:</strong> عاصف <strong>number_of_cars:</strong> 355 <strong>average_car_speed:</strong> 23.02 <strong>traffic_status:</strong>  مزدحم <strong>day_name:</strong> الخميس <strong>date:</strong> None <strong>point_geometry_wkt:</strong> POINT (46.7578008 24.7122812) <strong>geometry:</strong> LINESTRING (46.7583241 24.7126064, 46.7578008 24.7122812, 46.7567208 24.7116006)</p>\n\n<h1>لايوجد ازدحام</h1>\n<p><strong>accident_id:</strong> 133 <strong>street_name_left:</strong> مكة المكرمة <strong>number_of_cars_involved:</strong> 1 <strong>time_se

In [33]:
print(f'\n\nQuestion:\n{response2["question"]}\n\nAnswer: \n{response2["text"]}')



Question:
what is most accident reason ?

Answer: 
من خلال تحليل البيانات، نجد أن أكثر الأسباب لحدوث الحوادث هي "استخدام الهاتف"، والتي ظهرت 6 مرات في البيانات.


In [34]:
response = query_rag("what is most street name have traffic")
response



> Entering new LLMChain chain...
Prompt after formatting:

السياق: <h1>مزدحم</h1>
<p><strong>accident_id:</strong> 315 <strong>street_name_left:</strong> مكة المكرمة <strong>number_of_cars_involved:</strong> 2 <strong>time_series_left:</strong> 2023-02-21 12:00:00 <strong>accident_reason:</strong> السرعة <strong>number_of_injured:</strong> 0 <strong>traffic_id:</strong> 269857 <strong>time_series_right:</strong> 2023-02-21 12:00:00 <strong>street_name_right:</strong> مكة المكرمة <strong>weather:</strong> ضبابي <strong>number_of_cars:</strong> 247 <strong>average_car_speed:</strong> 33.74 <strong>traffic_status:</strong>  مزدحم <strong>day_name:</strong> الثلاثاء <strong>date:</strong> None <strong>point_geometry_wkt:</strong> POINT (46.6796404 24.6754022) <strong>geometry:</strong> LINESTRING (46.6789503 24.6752824, 46.6796404 24.6754022, 46.6800193 24.6754698)</p>

<h1>مزدحم</h1>
<p><strong>accident_id:</strong> 1000 <strong>street_name_left:</strong> مكة المكرمة <strong>number_of_c

{'context': '<h1>مزدحم</h1>\n<p><strong>accident_id:</strong> 315 <strong>street_name_left:</strong> مكة المكرمة <strong>number_of_cars_involved:</strong> 2 <strong>time_series_left:</strong> 2023-02-21 12:00:00 <strong>accident_reason:</strong> السرعة <strong>number_of_injured:</strong> 0 <strong>traffic_id:</strong> 269857 <strong>time_series_right:</strong> 2023-02-21 12:00:00 <strong>street_name_right:</strong> مكة المكرمة <strong>weather:</strong> ضبابي <strong>number_of_cars:</strong> 247 <strong>average_car_speed:</strong> 33.74 <strong>traffic_status:</strong>  مزدحم <strong>day_name:</strong> الثلاثاء <strong>date:</strong> None <strong>point_geometry_wkt:</strong> POINT (46.6796404 24.6754022) <strong>geometry:</strong> LINESTRING (46.6789503 24.6752824, 46.6796404 24.6754022, 46.6800193 24.6754698)</p>\n\n<h1>مزدحم</h1>\n<p><strong>accident_id:</strong> 1000 <strong>street_name_left:</strong> مكة المكرمة <strong>number_of_cars_involved:</strong> 1 <strong>time_series_left:</

In [35]:
print(f'\n\nQuestion:\n{response["question"]}\n\nAnswer: \n{response["text"]}')



Question:
what is most street name have traffic

Answer: 
من خلال تحليل البيانات، نجد أن أكثر الشوارع التي لديها تrafic هو "مكة المكرمة" والتي تظهر في 8 من 10 حالات من البيانات الم 제공ة.
